In [1]:
import pandas as pd
import numpy as np

In [2]:
df0= pd.read_parquet('../data/raw/transactions_small/part-00000-1f9c4e6c-eba1-48de-a950-1fde85d9c5b6-c000.snappy.parquet')
df1= pd.read_parquet('../data/raw/transactions_small/part-00001-1f9c4e6c-eba1-48de-a950-1fde85d9c5b6-c000.snappy.parquet')
df2= pd.read_parquet('../data/raw/transactions_small/part-00002-1f9c4e6c-eba1-48de-a950-1fde85d9c5b6-c000.snappy.parquet')
df3= pd.read_parquet('../data/raw/transactions_small/part-00003-1f9c4e6c-eba1-48de-a950-1fde85d9c5b6-c000.snappy.parquet')
df4= pd.read_parquet('../data/raw/transactions_small/part-00004-1f9c4e6c-eba1-48de-a950-1fde85d9c5b6-c000.snappy.parquet')
df5= pd.read_parquet('../data/raw/transactions_small/part-00005-1f9c4e6c-eba1-48de-a950-1fde85d9c5b6-c000.snappy.parquet')
df6= pd.read_parquet('../data/raw/transactions_small/part-00006-1f9c4e6c-eba1-48de-a950-1fde85d9c5b6-c000.snappy.parquet')
df7= pd.read_parquet('../data/raw/transactions_small/part-00007-1f9c4e6c-eba1-48de-a950-1fde85d9c5b6-c000.snappy.parquet')
df8= pd.read_parquet('../data/raw/transactions_small/part-00008-1f9c4e6c-eba1-48de-a950-1fde85d9c5b6-c000.snappy.parquet')
df9= pd.read_parquet('../data/raw/transactions_small/part-00009-1f9c4e6c-eba1-48de-a950-1fde85d9c5b6-c000.snappy.parquet')

df_transactions = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]).reset_index(drop=True).drop_duplicates()

In [3]:
df_world= pd.read_parquet('../data/raw/world_data/part-00000-1ffc4f17-2dd5-4f08-94aa-313697ef63d2-c000.snappy.parquet')

In [4]:
df_securit = pd.read_parquet('../data/raw/securities/part-00000-8d4b72c8-6d72-4b4d-9f02-4d030bdfad38-c000.snappy.parquet')

In [5]:
df_daily = pd.read_parquet('../data/raw/daily_quotation/part-00000-1de77bcf-ec55-4981-bd56-3cf03c412272-c000.snappy.parquet')

In [6]:
df_accounts = pd.read_parquet('../data/raw/accounts/part-00000-af066738-0f3b-45e4-a08f-6d7ed4f84424-c000.snappy.parquet')

In [7]:
df0= pd.read_parquet('../data/raw/forex_ex/part-00000-27bd45d4-da0e-47e7-b151-a47caedbc2e6-c000.snappy.parquet')
df1= pd.read_parquet('../data/raw/forex_ex/part-00001-27bd45d4-da0e-47e7-b151-a47caedbc2e6-c000.snappy.parquet')
df2= pd.read_parquet('../data/raw/forex_ex/part-00002-27bd45d4-da0e-47e7-b151-a47caedbc2e6-c000.snappy.parquet')
df3= pd.read_parquet('../data/raw/forex_ex/part-00003-27bd45d4-da0e-47e7-b151-a47caedbc2e6-c000.snappy.parquet')
df4= pd.read_parquet('../data/raw/forex_ex/part-00004-27bd45d4-da0e-47e7-b151-a47caedbc2e6-c000.snappy.parquet')


df_forex = pd.concat([df0, df1, df2, df3, df4]).reset_index(drop=True).drop_duplicates()

In [8]:

def safe_div(a: pd.Series, b: pd.Series):
    return np.divide(a, b,out=np.full_like(a, np.nan, dtype=np.float64), where=(b!=0) & (~pd.isna(b)))

In [9]:
def normalize_date(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s, errors='coerce').dt.normalize()

In [10]:
df_transactions.columns

Index(['receive_security', 'deliver_security', 'transaction_timestamp', 'isin',
       'unit_delivered', 'market_value', 'currency_code'],
      dtype='object')

In [11]:
tx = (
    df_transactions.copy()
    .rename(columns={'currency_code': 'tx_currency_code'})
    .assign(transaction_date=normalize_date(df_transactions['transaction_timestamp']))
)

In [12]:
tx.head()

,receive_security,deliver_security,transaction_timestamp,isin,unit_delivered,market_value,tx_currency_code,transaction_date
0,933,997,2024-11-18 21:47:58.867,NRZRPI76DSXX,97,10647.076029,AUD,2024-11-18
1,290,786,2024-04-21 05:53:50.696,SA1GIUE3SGXX,55,2750.000000,SAR,2024-04-21
2,831,856,2024-03-22 04:45:16.139,GBQE9FJGE0XX,33,3372.971297,GBP,2024-03-22
3,999,786,2024-07-30 22:13:18.713,IN9EY3ND49XX,67,6514.324093,INR,2024-07-30
4,308,963,2024-05-23 22:00:03.077,PYOGGU527EXX,26,2644.395586,PYG,2024-05-23


In [13]:
df_sec = df_securit.copy().rename(columns={'currency_code': 'security_currency_code'}).drop_duplicates(subset=['isin'])


In [14]:
tx = tx.merge(df_sec, on='isin', how='left', validate='m:1')

In [15]:
tx

,receive_security,deliver_security,transaction_timestamp,isin,unit_delivered,market_value,tx_currency_code,transaction_date,security_currency_code,instrument_type,issuer,issuer_country_code
0,933,997,2024-11-18 21:47:58.867,NRZRPI76DSXX,97,10647.076029,AUD,2024-11-18,AUD,BOND,"Denesik, Harris and Thiel",NR
1,290,786,2024-04-21 05:53:50.696,SA1GIUE3SGXX,55,2750.000000,SAR,2024-04-21,SAR,BOND,Powlowski Inc,SA
2,831,856,2024-03-22 04:45:16.139,GBQE9FJGE0XX,33,3372.971297,GBP,2024-03-22,GBP,BOND,Predovic Inc,GB
3,999,786,2024-07-30 22:13:18.713,IN9EY3ND49XX,67,6514.324093,INR,2024-07-30,INR,BOND,"Franecki, Thiel and Goldner",IN
4,308,963,2024-05-23 22:00:03.077,PYOGGU527EXX,26,2644.395586,PYG,2024-05-23,PYG,BOND,Cummerata-Rolfson,PY
...,...,...,...,...,...,...,...,...,...,...,...,...
415512,398,855,2024-02-21 14:47:44.318,SZ37RQ6195XX,330,32432.870825,SZL,2024-02-21,SZL,BOND,Romaguera-MacGyver,SZ
415513,856,398,2024-03-28 17:33:08.640,OMQ1Q77F6HXX,156,15537.343334,OMR,2024-03-28,OMR,BOND,"Lebsack, Grady and Aufderhar",OM
415514,999,326,2024-10-06 19:48:29.840,AEJGLFAYWMXX,1275,63750.000000,AED,2024-10-06,AED,BOND,"Reinger, Lueilwitz and Ankunding",AE
415515,398,772,2024-08-24 16:55:53.070,BSLYJ56135XX,448,22400.000000,BSD,2024-08-24,BSD,BOND,"Blanda, Reichert and Grimes",BS


In [16]:
df_ex = df_forex.copy().rename(columns={'equivalent_eur': 'tx_equicalent_eur'}).drop_duplicates(subset=['currency_code'])

In [17]:
tx = (
    tx.merge(df_ex, left_on='tx_currency_code', right_on='currency_code', how='left', validate='m:1')
    .drop(columns=['currency_code'])
)

In [18]:
tx.columns

Index(['receive_security', 'deliver_security', 'transaction_timestamp', 'isin',
       'unit_delivered', 'market_value', 'tx_currency_code',
       'transaction_date', 'security_currency_code', 'instrument_type',
       'issuer', 'issuer_country_code', 'tx_equicalent_eur'],
      dtype='object')

In [19]:
tx['market_value_eur'] =  safe_div(tx['market_value'], tx['tx_equicalent_eur'])

In [20]:
df_daily

,calendar_date,isin,real_quotation
0,2024-01-02,AD2711IS7XXX,100.000000
1,2024-01-03,AD2711IS7XXX,99.885397
2,2024-01-04,AD2711IS7XXX,99.752717
3,2024-01-05,AD2711IS7XXX,100.097947
4,2024-01-08,AD2711IS7XXX,100.017189
...,...,...,...
492202,2024-12-20,ZWXEALYZJ8XX,94.635349
492203,2024-12-23,ZWXEALYZJ8XX,94.532868
492204,2024-12-26,ZWXEALYZJ8XX,94.496609
492205,2024-12-27,ZWXEALYZJ8XX,93.950372


In [21]:

df_daily['calendar_date'] = normalize_date(df_daily['calendar_date'])

In [22]:
tx = tx.merge(df_daily, left_on=['isin','transaction_date'], right_on=['isin', 'calendar_date'],  how='left', validate='m:1')

In [23]:
tx

,receive_security,deliver_security,transaction_timestamp,isin,unit_delivered,market_value,tx_currency_code,transaction_date,security_currency_code,instrument_type,issuer,issuer_country_code,tx_equicalent_eur,market_value_eur,calendar_date,real_quotation
0,933,997,2024-11-18 21:47:58.867,NRZRPI76DSXX,97,10647.076029,AUD,2024-11-18,AUD,BOND,"Denesik, Harris and Thiel",NR,9.533440,1116.813657,2024-11-18,109.763670
1,290,786,2024-04-21 05:53:50.696,SA1GIUE3SGXX,55,2750.000000,SAR,2024-04-21,SAR,BOND,Powlowski Inc,SA,5.461113,503.560328,NaT,NaN
2,831,856,2024-03-22 04:45:16.139,GBQE9FJGE0XX,33,3372.971297,GBP,2024-03-22,GBP,BOND,Predovic Inc,GB,9.472152,356.093459,2024-03-22,102.211251
3,999,786,2024-07-30 22:13:18.713,IN9EY3ND49XX,67,6514.324093,INR,2024-07-30,INR,BOND,"Franecki, Thiel and Goldner",IN,8.209380,793.522040,2024-07-30,97.398087
4,308,963,2024-05-23 22:00:03.077,PYOGGU527EXX,26,2644.395586,PYG,2024-05-23,PYG,BOND,Cummerata-Rolfson,PY,1.792145,1475.547403,2024-05-23,101.525409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415512,398,855,2024-02-21 14:47:44.318,SZ37RQ6195XX,330,32432.870825,SZL,2024-02-21,SZL,BOND,Romaguera-MacGyver,SZ,2.562812,12655.188667,2024-02-21,98.281427
415513,856,398,2024-03-28 17:33:08.640,OMQ1Q77F6HXX,156,15537.343334,OMR,2024-03-28,OMR,BOND,"Lebsack, Grady and Aufderhar",OM,3.201145,4853.683249,2024-03-28,99.598355
415514,999,326,2024-10-06 19:48:29.840,AEJGLFAYWMXX,1275,63750.000000,AED,2024-10-06,AED,BOND,"Reinger, Lueilwitz and Ankunding",AE,6.033130,10566.654491,NaT,NaN
415515,398,772,2024-08-24 16:55:53.070,BSLYJ56135XX,448,22400.000000,BSD,2024-08-24,BSD,BOND,"Blanda, Reichert and Grimes",BS,3.376701,6633.692688,NaT,NaN


In [24]:
tx = tx.drop(columns=['calendar_date'])

In [25]:
df_world.columns

Index(['country', 'country_code', 'currency_name', 'currency_code',
       'country_risk'],
      dtype='object')

In [26]:
df_accounts.columns

Index(['account_id', 'company_name', 'address', 'country'], dtype='object')

In [27]:
tx.columns

Index(['receive_security', 'deliver_security', 'transaction_timestamp', 'isin',
       'unit_delivered', 'market_value', 'tx_currency_code',
       'transaction_date', 'security_currency_code', 'instrument_type',
       'issuer', 'issuer_country_code', 'tx_equicalent_eur',
       'market_value_eur', 'real_quotation'],
      dtype='object')

In [28]:
df_country_issuer = df_world.rename(
    columns={'country': 'issuer_country', 'country_risk': 'issuer_country_risk', 'country_code': 'issuer_country_code'}
)[['issuer_country_code', 'issuer_country', 'issuer_country_risk']].drop_duplicates(subset='issuer_country_code')

In [29]:
tx = tx.merge(
    df_country_issuer,
    on = 'issuer_country_code',
    how= 'left',
    validate='m:1'
)

In [30]:
tx.shape

(415517, 17)

In [31]:
df_acc = df_accounts.rename(columns={'country': 'account_country'}).copy()

In [32]:
df_acc 

,account_id,company_name,address,account_country
0,844,Stark-Wilkinson,"11959 Andrew Mills,North Raleigh",Afghanistan
1,720,"Spencer, Vandervort and Jacobs","28934 Schaden Locks,Desiraetown",Afghanistan
2,49,Ziemann-Lowe,"09596 Jesus Mission,South Sana",Afghanistan
3,39,"O'Hara, MacGyver and Ratke","176 Everett Radial,North Ulysses",Afghanistan
4,864,Jakubowski Inc,"395 Smith Course,South Blaketon",Albania
...,...,...,...,...
995,2,Kassulke-Breitenberg,"5440 Prosacco Hollow,Renatoberg",Zambia
996,528,Senger Inc,"71666 Lon Streets,Janellaton",Zimbabwe
997,494,Brown-Mueller,"750 Wehner Trace,Benview",Zimbabwe
998,489,Gutmann-Wisozk,"1088 Lebsack Vista,Richiefort",Zimbabwe


In [33]:
df_receive = df_acc.rename(
        columns={
            "account_id": "receive_account_id",
            "company_name": "receive_company",
            "address": "receive_address",
            "account_country": "receive_country",
        }
)[['receive_account_id', 'receive_company', 'receive_address', 'receive_country']].drop_duplicates(subset='receive_account_id')

In [34]:
tx = tx.merge(
    df_receive,
    left_on="receive_security",      # sprawdź, czy ta kolumna istnieje w tx
    right_on="receive_account_id",
    how="left",
    validate="m:1"
)

In [35]:
tx.shape

(415517, 21)

In [36]:
df_deliver = (
    df_acc.rename(
        columns={
            "account_id": "deliver_account_id",
            "company_name": "deliver_company",
            "address": "deliver_address",
            "account_country": "deliver_country",
        }
    )[["deliver_account_id", "deliver_company", "deliver_address", "deliver_country"]]
    .drop_duplicates(subset="deliver_account_id")
)

tx = tx.merge(
    df_deliver,
    left_on="deliver_security",      # sprawdź, czy ta kolumna istnieje w tx
    right_on="deliver_account_id",
    how="left",
    validate="m:1"
)

In [38]:
tx.columns

Index(['receive_security', 'deliver_security', 'transaction_timestamp', 'isin',
       'unit_delivered', 'market_value', 'tx_currency_code',
       'transaction_date', 'security_currency_code', 'instrument_type',
       'issuer', 'issuer_country_code', 'tx_equicalent_eur',
       'market_value_eur', 'real_quotation', 'issuer_country',
       'issuer_country_risk', 'receive_account_id', 'receive_company',
       'receive_address', 'receive_country', 'deliver_account_id',
       'deliver_company', 'deliver_address', 'deliver_country'],
      dtype='object')

In [39]:
tx = tx.merge(
    df_world.rename(
        columns={
            'country_code': 'receive_country_code',
            'country_risk': 'receive_country_risk',
            'country': 'receive_country'
        }
    )[['receive_country', 'receive_country_code', 'receive_country_risk']].drop_duplicates(subset='receive_country'),
    on='receive_country',
    how='left',
    validate='m:1'
)

In [41]:
tx = tx.merge(
    df_world.rename(
        columns={
            'country_code': 'deliver_country_code',
            'country_risk': 'deliver_country_risk',
            'country': 'deliver_country'
        }
    )[['deliver_country', 'deliver_country_code', 'deliver_country_risk']].drop_duplicates(subset='deliver_country'),
    on='deliver_country',
    how='left',
    validate='m:1'
)

In [42]:
tx.columns

Index(['receive_security', 'deliver_security', 'transaction_timestamp', 'isin',
       'unit_delivered', 'market_value', 'tx_currency_code',
       'transaction_date', 'security_currency_code', 'instrument_type',
       'issuer', 'issuer_country_code', 'tx_equicalent_eur',
       'market_value_eur', 'real_quotation', 'issuer_country',
       'issuer_country_risk', 'receive_account_id', 'receive_company',
       'receive_address', 'receive_country', 'deliver_account_id',
       'deliver_company', 'deliver_address', 'deliver_country',
       'receive_country_code', 'receive_country_risk', 'deliver_country_code',
       'deliver_country_risk'],
      dtype='object')

In [43]:
preferred = [
        "transaction_timestamp", "transaction_date", "isin",
        "unit_delivered", "market_value", "market_value_eur",
        "tx_currency_code", "tx_equivalent_eur", "real_quotation",
        "security_currency_code", "instrument_type", "issuer",
        "issuer_country_code", "issuer_country", "issuer_country_risk",
        "receive_security", "receive_account_id", "receive_company",
        "receive_address", "receive_country", "receive_country_code", "receive_country_risk",
        "deliver_security", "deliver_account_id", "deliver_company",
        "deliver_address", "deliver_country", "deliver_country_code", "deliver_country_risk",
    ]

In [44]:
tx.shape

(415517, 29)

In [45]:
cols = [c for c in preferred if c in tx.columns] + [c for c in tx.columns if c not in preferred]
tx.loc[:, cols].to_parquet('../data/interim/tx.parquet')

In [46]:
tx['unite_price_tx'] = safe_div(tx['market_value'], tx['unit_delivered'])

In [47]:
tx['unit_price_eur'] = safe_div(tx['market_value_eur'], tx['unit_delivered'])

In [48]:
tx['weekday'] = tx['transaction_date'].dt.dayofweek
tx['is_weekend'] = tx['weekday'] >= 5

In [49]:
tx.head()

,receive_security,deliver_security,transaction_timestamp,isin,unit_delivered,market_value,tx_currency_code,transaction_date,security_currency_code,instrument_type,...,deliver_address,deliver_country,receive_country_code,receive_country_risk,deliver_country_code,deliver_country_risk,unite_price_tx,unit_price_eur,weekday,is_weekend
0,933,997,2024-11-18 21:47:58.867,NRZRPI76DSXX,97,10647.076029,AUD,2024-11-18,AUD,BOND,...,"1340 Monahan Union,Gerlachstad",Saint Lucia,FI,1,LC,1,109.763670,11.513543,0,False
1,290,786,2024-04-21 05:53:50.696,SA1GIUE3SGXX,55,2750.000000,SAR,2024-04-21,SAR,BOND,...,"3166 Herman Bridge,New Lindsey",Vanuatu,AU,1,VU,1,50.000000,9.155642,6,True
2,831,856,2024-03-22 04:45:16.139,GBQE9FJGE0XX,33,3372.971297,GBP,2024-03-22,GBP,BOND,...,"6789 Carroll Shores,East Alfredmouth",Switzerland,AE,2,CH,1,102.211251,10.790711,4,False
3,999,786,2024-07-30 22:13:18.713,IN9EY3ND49XX,67,6514.324093,INR,2024-07-30,INR,BOND,...,"3166 Herman Bridge,New Lindsey",Vanuatu,CN,1,VU,1,97.228718,11.843613,1,False
4,308,963,2024-05-23 22:00:03.077,PYOGGU527EXX,26,2644.395586,PYG,2024-05-23,PYG,BOND,...,"307 Stark Forks,Herthaborough",Sweden,MH,1,SE,1,101.707523,56.751823,3,False
